# Bayesian Camera Calibration
> Let's look and how we can use PyMC3 to help us improve our aim.

- toc: true 
- badges: true
- comments: true
- categories: [Bayesian, PyMC3]
- image: images/2020-07-02-On-Target-With-PyMC3/header.jpg

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pymc3 as pm
import theano
import pandas as pd
from scipy.spatial.transform import Rotation as Rot
from warnings import filterwarnings
filterwarnings('ignore')

plt.rcParams['figure.figsize'] = [10,10]

In [2]:
df = pd.read_csv('data/2020-07-05-Bayesian-Camera-Calibration/points.csv',sep =' ')
px = df.i.values
py = df.j.values

X_input = df.X.values
Y_input = df.Y.values
Z_input = df.Z.values

number_points = px.shape[0]

points3d = np.vstack([X_input,Y_input,Z_input]).T

In [3]:
def Rotate_Translate(X_est, Y_est, Z_est):
    #Quaternions: X: 0.852 Y: -0.242 Z: 0.139 W: 0.442 
    Q0 = pm.Normal('Wq', mu = 0.166,  sigma=0.1)
    Q1 = pm.Normal('Xq', mu = 0.894,  sigma=0.1)
    Q2 = pm.Normal('Yq', mu = -0.408, sigma=0.1)
    Q3 = pm.Normal('Zq', mu = 0.084,  sigma=0.1)
    
    norm = pm.math.sqrt(Q0**2 + Q1**2 + Q2**2 + Q3**2)
    
    Q0 /= norm
    Q1 /= norm
    Q2 /= norm
    Q3 /= norm
    
    R =[[Q0**2 + Q1**2 - Q2**2 - Q3**2, 2*(Q1*Q2 - Q0*Q3), 2*(Q0*Q2 + Q1*Q3)],
        [2*(Q1*Q2 + Q0*Q3), Q0**2 - Q1**2 + Q2**2 - Q3**2, 2*(Q2*Q3 - Q0*Q1)],
        [2*(Q1*Q3 - Q0*Q2), 2*(Q0*Q1 + Q2*Q3), (Q0**2 - Q1**2 - Q2**2 + Q3**2)]]
    
    
    # Define priors 
    X_translate = pm.Normal('X_translate', mu = -6.85, sigma = 5)
    Y_translate = pm.Normal('Y_translate', mu = -12.92, sigma = 5)
    Z_translate = pm.Normal('Z_translate', mu = 2.75, sigma = 5)
    
    RIC_0_3 = R[0][0] * -X_translate + R[0][1] * -Y_translate + R[0][2] * -Z_translate
    RIC_1_3 = R[1][0] * -X_translate + R[1][1] * -Y_translate + R[1][2] * -Z_translate
    RIC_2_3 = R[2][0] * -X_translate + R[2][1] * -Y_translate + R[2][2] * -Z_translate
    
    X_out = X_est * R[0][0] + Y_est * R[0][1] + Z_est * R[0][2] + RIC_0_3
    Y_out = X_est * R[1][0] + Y_est * R[1][1] + Z_est * R[1][2] + RIC_1_3
    Z_out = X_est * R[2][0] + Y_est * R[2][1] + Z_est * R[2][2] + RIC_2_3
    
    return(X_out, Y_out, Z_out)
    

In [ ]:
with pm.Model() as model: # model specifications in PyMC3 are wrapped in a with-statement
    X, Y, Z = Rotate_Translate(points3d[:,0], points3d[:,1], points3d[:,2])
    
    #Focal length (Pixels): 1010
    focal_length = pm.Normal('focal_length',mu = 1010, sigma = 100)
    
    k1 = pm.Normal('k1', mu = -0.327041, sigma = 0.01 * 0.327041)
    k2 = pm.Normal('k2', mu = 0.175031,  sigma = 0.01 * 0.175031)
    k3 = pm.Normal('k3', mu = -0.030751, sigma = 0.01 * 0.030751)
    
    c_x = pm.Normal('c_x', mu = 1038.58, sigma = 200)
    c_y = pm.Normal('c_y', mu = 2663.52, sigma = 200)
    
    px_est = X / Z
    py_est = Y / Z
    
    #Radial distortion
    r = pm.math.sqrt(px_est**2 + py_est**2)
    
    radial_distortion_factor = (1 + k1 * r + k2 * r**2 + k3 * r**3)
    px_est *= radial_distortion_factor
    py_est *= radial_distortion_factor
    
    px_est *= focal_length
    py_est *= focal_length

    px_est += c_x
    py_est += c_y
    
    error_scale = 5 #px
    
    delta = pm.math.sqrt((px - px_est)**2 + (py - py_est)**2)
    
    # Define likelihood
    likelihood = pm.Normal('rms_pixel_error', mu = delta, sigma = error_scale, observed=np.zeros(number_points))

    # Inference!
    trace = pm.sample(draws=1_000, init='adapt_diag', cores=4 , tune=1_000)

Auto-assigning NUTS sampler...
Initializing NUTS using adapt_diag...
Multiprocess sampling (4 chains in 4 jobs)
NUTS: [c_y, c_x, k3, k2, k1, focal_length, Z_translate, Y_translate, X_translate, Zq, Yq, Xq, Wq]
Sampling 4 chains, 0 divergences:  39%|███▉      | 3153/8000 [03:23<06:51, 11.77draws/s]

In [ ]:
# Profiling of the logp call
#model.profile(model.logpt).summary()

In [ ]:
plt.figure(figsize=(7, 7))
pm.traceplot(trace);
plt.tight_layout();

In [ ]:
pm.plot_posterior(trace);

In [ ]:
pm.summary(trace)

In [ ]:
pm.pairplot(trace, var_names=['X_translate','Y_translate','Z_translate'], divergences=True);

In [ ]:
pm.pairplot(trace, var_names=['k1', 'k2', 'k3'], divergences=True);

In [ ]:
pm.pairplot(trace, var_names=['c_x', 'c_y'], divergences=True);

In [ ]:
pm.pairplot(trace, var_names=['Wq', 'Xq','Yq','Zq'], divergences=True);

In [ ]:
sns.jointplot(trace[:]['X_translate'], trace[:]['Y_translate'], kind="hex");

In [ ]:
sns.jointplot(trace[:]['X_translate'], trace[:]['Z_translate'], kind="hex");

In [ ]:
sns.jointplot(trace[:]['c_x'], trace[:]['c_y'], kind="hex");

In [ ]:
sns.jointplot(trace[:]['Wq'], trace[:]['Xq'], kind="hex");